In [ ]:
# Copyright 2019 Institut für Nachrichtentechnik, RWTH Aachen University
%matplotlib widget

import sys

import ipywidgets as widgets
from IPython.display import clear_output, display, HTML
from IPython.display import Markdown as md

import numpy as np
from matplotlib.patches import ConnectionPatch
import matplotlib.pyplot as plt

from rwth_nb.plots import colors

import rwth_nb.plots.mpl_decorations as rwth_plt

# rwth_nb feedback
import rwth_nb.misc.feedback as rwth_feedback

def dreieckTime(start, end_d):
    peak=(end_d-start)//2
    left_side = np.arange(peak+1)/peak
    right_side = 1-left_side[1:]
    t=np.concatenate((left_side,right_side))
    return t
def SiFreq(freq):
    si_f = np.sinc(freq)**2
    return si_f
def position(pos,func,l):
    dist=(np.size(func)-1)//2
    array = np.zeros(l)
    start_p = pos-dist
    end_p=pos+dist+1
    start_f=0
    end_f=np.size(func)
    if start_p < 0:
        start_p=0
        start_f=np.absolute(pos-dist)
    if end_p > l:
        end_p = l
        end_f = end_p-start_p

    array[start_p:end_p]=func[start_f:end_f]
    return array

def set_label_right(ax, name):
    ax.set_xlabel(name )
    label = ax.xaxis.get_label()
    x_lab_pos, y_lab_pos = label.get_position()
    label.set_position([0.95, y_lab_pos])
    

<div><img src="http://www.ient.rwth-aachen.de/cms/uploads/images/rwth_ient_logo@2x.png" style="float: right; height: 5em;"></div>

# Vorbereitungsaufgaben Versuch 7

Zur Vorbereitung des Versuches müssen Sie zunächst das Vorbereitungsskript lesen. Anschließend bearbeiten Sie bitte die Inhaltlichen Fragen (Teil 1 der Vorbereitungsaufgaben) und machen sich ein wenig mit Python vertraut (Teil 2 der Vorbereitungsaufgaben).

## Teil 1: Inhaltliche Fragen

Bitte beantworten Sie die Fragen schriftlich und bringen Sie ihre Lösungen zum Versuch mit.


<div class="alert rwth-topic">

### Aufgabe 1.1: Verdeckung im Frequenzbereich

Gegeben ist ein Sinuston mit einer Frequenz von $1\,\mathrm{kHz}$ und einer Lautstärke von $70\,\mathrm{dB}$. Wie laut muss ein Sinuston mit der Frequenz $2\,\mathrm{kHz}$ mindestens sein, damit er vom menschlichen Gehör wahrgenommen werden kann?
    
</div>

<div class="alert rwth-topic">

### Aufgabe 1.2: Normalisieren

Gegeben sind zwei Signale $s_1[n]$ und $s_2 [n]$ mit der Energie $E_1$ bzw $E_2$. Mit welchem Faktor $a$ muss $s_2[ n]$ multipliziert werden, damit es die Zielenergie $E_1$ besitzt?
    
</div>

<div class="alert rwth-topic">

### Aufgabe 1.3: Diskrete Fourier-Transformation (DFT)
Zum Bearbeiten dieser Aufgabe benutzen Sie bitte die beiden Abbildungen "Abtastung im Zeit- und Frequenzbereich" und "Diskrete Fourier-Transformation (DFT)".

In der ersten Abbildung wird die Diskrete Fourier-Transformation durch Abtastung sowohl im Zeitbereich als auch im Frequenzbereich erklärt:

* In der oberen Hälfte der Abbildung sehen Sie ein Dreiecksignal $s(t)=\Lambda(t)$ in der unteren Hälfte das dazugehörige Spektrum $S(f)=\text{si}^2(f)$. 

* Am unteren Ende der Abbildung sehen Sie zwei Slider: 
 * Über den Slider $r$ können Sie die Abtastrate im Zeitbereich vorgeben
 * Über den Slider $r_f$ können Sie die Abtastrate im Frequenzbereich vorgeben

In der zweiten Abbildung mit dem Titel "Diskrete Fourier-Transformation (DFT)" wird auf ein diskretes Zeitsignal die DFT mithilfe der Funktion _np.fft.fft()_ angewendet. 

* In der oberen Hälfte der Abbildung sehen Sie ein mit $r =4\,\mathrm{Hz}$ abgetastetes Dreiecksignal, in der unteren Hälfte das mit der DFT erzeugte Spektrum $S_{\mathrm{d}}(k)$. Beachten Sie die Achsenbeschriftungen jeweils oben und unten 

* Am unteren Ende der Abbildung sehen Sie einen Slider, mit dem die Transformationslänge $N$ eingestellt werden kann.

__Bitte beantworten Sie die folgenden Fragen:__

1. Was passiert im Frequenzbereich, wenn Sie das Signal im Zeitbereich abtasten? Was muss für die Abtastrate $r$ gelten, damit das abgetastete Signal fehlerfrei rekonstruiert werden kann?

2. Was passiert im Zeitbereich, wenn Sie das Spektrum im Frequenzbereich abtasten? Was muss für die Abtastrate $r_f$ gelten, damit das abgetastete Spektrum fehlerfrei rekonstruiert werden kann?

3. Was passiert, wenn in beiden Bereichen abgetastet wird? Was bedeutet der orangene Kasten?

4. Wie berechnet sich die Transformationslänge $N$?

5. Wie berechnet sich die Grundfrequenz $f_\Delta$?

6. Stellen Sie in der ersten Abbildung "Abtastung im Zeit- und Frequenzbereich" die Slider auf $r=4\,\mathrm{Hz}$ und $r_f=4\,\mathrm{Hz}$. Schauen Sie sich nun die zweite Abbildung "Diskrete Fourier-Transformation (DFT)" an. Was können Sie feststellen?
</div>

In [ ]:
#figure and subplot properties, two x-Achses
fig= plt.figure(num='Abtastung im Zeit- und Frequenzbereich', figsize=(10,6))
fig.subplots_adjust(hspace = 0.6,top=0.88,bottom=0.1)

ax_t = fig.add_subplot(2, 1,1)
ax_f = fig.add_subplot(2, 1,2)

ax_n = ax_t.twiny()
ax_k = ax_f.twiny()



#constants for time and frequency domain

time_delta = np.array([40,40,40,42,40,40])
time_start=-5
time_end=10
time_total = time_end-(time_start)

freq_delta = np.array([20,20,20,18,24,20])
freq_end=10
freq_start=-5
freq_total = freq_end-(freq_start)



def update_plot(r,r_f): 
    
    #cleanig of figure
    print('')
    
    ax_t.clear()
    ax_f.clear()

    ax_n.clear()
    ax_k.clear()
        
    ax_n.set_xlim([time_start,time_end]) #for non sampled signal set diskrete axis equivalent to continous axis
    ax_k.set_xlim([freq_start,freq_end])   
    
    
    ax_t.set_title('Zeitbereich')
    ax_f.set_title('Frequenzbereich')
    
    # x-axis time and frequency -l_t/l_f addopt to the samplingrate -for even sampling
    l_t= time_delta[r//2]*time_total+1
    l_f=freq_delta[r_f//2]*freq_total+1

    time = np.linspace(time_start,time_end,l_t)
    freq = np.linspace(freq_start,freq_end,l_f)

    # signal time and frequency domain
    d_t = dreieckTime(0,time_delta[r//2]*2)
    si_f = SiFreq(freq[0:2*freq_delta[r_f//2]*np.absolute(freq_start)+1])
    
    #Generates the continous signals (time and frequency spectrum)
    S_f = position(freq_delta[r_f//2]*np.absolute(freq_start),si_f,l_f) 
    s_t = position(time_delta[r//2]*np.absolute(time_start),d_t,l_t)

    
    if r_f!=0:
        
        #Determine the number of copies on each side
        l_shift_f = np.absolute(time_start)//r_f
        if np.mod(np.absolute(time_start),r_f)>(r_f/2-1):#halbereiecksbreite im Zeitbereich =1
            l_shift_f +1    
        r_shift_f = np.absolute(time_end)//r_f
        if np.mod(np.absolute(time_end),r_f)>(r_f/2-1):#halbereiecksbreite im Zeitbereich =1
            r_shift_f +1
            
        #Generate the periodic copies in the time domain, using a summation over shifted originals,
        #the first s_t already contains the signal around 0s
        a = np.concatenate((np.arange(l_shift_f),np.arange(l_shift_f+1,l_shift_f+r_shift_f+1)))
        for x in a:
            s_t = position((x-l_shift_f)*r_f*time_delta[r//2] + (time_delta[r//2]*np.absolute(time_start)), d_t, l_t) + s_t
        
        #generate the sampled version of the frequency spectrum
        S_d = S_f[np.arange(0,l_f,freq_delta[r_f//2]//r_f)]
        
        #Design & plot - discrete frequence domain: legend, axislabes and limits 
        rwth_plt.stem(ax_k, np.arange(np.size(S_d)), S_d, color='rwth:blue', markerfmt="o", label= '$S(k)$')
        ax_k.set_xlim([0,np.size(S_d)-1]) #garantees alignment of continous and diskrete)
        ax_k.legend(loc='upper right', bbox_to_anchor=(1.0, 0.8))
        set_label_right(ax_k,r'$\rightarrow k$')
        ax_k.xaxis.set_label_coords(1.02, 1)

        
        
    if r!=0:
        #Determine the number of copies on each side 
        # !!! becareful it only works that easx because the size of the frequency spectun is 2*5 !!!        
        l_shift= np.int(np.rint(np.absolute(freq_start)/r))
        r_shift= np.int(np.rint(freq_end/r))
        
        #Generate the periodic copies in the frequency domain, using a summation over shifted originals,
        #the first S_f already contains the spectrum around 0Hz
        a = np.concatenate((np.arange(l_shift),np.arange(l_shift+1,l_shift+r_shift+1)))
        for x in a:
            S_f = position((x-l_shift)*r*freq_delta[r_f//2] + (freq_delta[r_f//2]*np.absolute(freq_start)), si_f, l_f) + S_f
        
        #generate the sampled version of the time signal        
        s_d = s_t[np.arange(0,l_t,time_delta[r//2]//r)]
        
        #Design & plot - discrete time domain: legend, axislabes and limits 
        rwth_plt.stem(ax_n,np.arange(np.size(s_d)),s_d, label= '$s(n)$')
        ax_n.set_xlim([0,np.size(s_d)-1]) #garantees alignment of continous and diskrete)
        ax_n.legend(loc='upper right', bbox_to_anchor=(1.0, 0.8))
        set_label_right(ax_n,r'$\rightarrow n$')
        ax_n.xaxis.set_label_coords(1.02, 1)

        
     
    if r!=0 and r_f!=0:
        #generate the sampled version of the frequency spectrum
        S_d = S_f[np.arange(0,l_f,freq_delta[r_f//2]//r_f)]
        
        #Design & plot - discrete frequence domain: legend, axislabes and limits 
        ax_k.clear()
        rwth_plt.stem(ax_k, np.arange(np.size(S_d)), S_d, color='rwth:blue', markerfmt="o", label= '$S(k)$')
        ax_k.legend(loc='upper right', bbox_to_anchor=(1.0, 0.8) )
        ax_k.set_xlim([0, np.size(S_d) - 1])
        set_label_right(ax_k, r'$\rightarrow k$')   
        ax_k.xaxis.set_label_coords(1.02, 1)

        
        #calculate and print transformation length
        N = r*r_f
        print(r'Transformationslänge N:')
        print(N)
                
        #draw delta f 
        string0="$f_\Delta$"
        ax_f.annotate(string0,xy=(0, 0.22), xycoords='data',color = 'orange')
        con0 = ConnectionPatch(xyA=(0,0.2), xyB=(r/N,0.2), coordsA="data", coordsB="data",
                      axesA=ax_f, axesB=ax_f, color = 'orange')
        ax_f.add_artist(con0)  
        
        #draw one periode
        ax_f.fill_between(np.arange(0,r+1)-0.4*(r/N), -0.1,1.1, 
                          facecolor="none", edgecolor='orange', linewidth=2.0) 
        ax_t.fill_between(np.arange(-r_f//2,r_f//2+1)-0.4*(r_f/N), -0.1,1.1, 
                          facecolor="none", edgecolor='orange', linewidth=2.0)  

    #Design & plot - continous time and frequency domain: legend, axislabes and limits
    ax_t.plot(time,s_t, label='$s(t)$')
    ax_f.plot(freq,S_f, label='$S(f)$')
    
    set_label_right(ax_t, r'$\rightarrow t $ in $[s]$')
    ax_t.xaxis.set_label_coords(1.04, 0)

    set_label_right(ax_f, r'$\rightarrow f $ in $[Hz]$')
    ax_f.xaxis.set_label_coords(1.04, 0)

    ax_t.set_xlim([time_start,time_end])
    ax_f.set_xlim([freq_start,freq_end])
    ax_t.legend(loc='upper right') 
    ax_f.legend(loc='upper right')


r = widgets.IntSlider(min = 0, max = 10, step = 2, value = 0, description = '$r \,\mathrm{[Hz]}$:')
r_f = widgets.IntSlider(min = 0, max = 10, step = 2, value = 0,  description = '$r_f \,\mathrm{[Hz]}$:')
    
widgets.interactive(update_plot, r=r,r_f=r_f)

In [ ]:
#figure and subplot properties, two x-Achses
fig1= plt.figure(num='Diskrete Fourier-Transfomation (DFT)', figsize=(10,6))
fig1.subplots_adjust(hspace = 0.6,top=0.88,bottom=0.1)

ax_n1 = fig1.add_subplot(2, 1,1)
ax_k1 = fig1.add_subplot(2, 1,2)

ax_t1 = ax_n1.twiny()
ax_f1 = ax_k1.twiny()

#constants for time domain
time_delta1 = np.array([40,40,40,42,40,40])
time_start1=0

#the ssampling frequency can besimply integrated as a slider - change: def update_plot(r1,N1) and 
#widgets.interactive(update_plot,r1=r1, N1=N1) an uncommment the r1 - widgets.IntSlider and comment r1=4!
r1=4

def update_plot(N1): 
    
    #cleanig of figure
    ax_n1.clear()
    ax_k1.clear()
    
    ax_t1.clear()
    ax_f1.clear()
    
    ax_n1.set_title('Zeitbereich')
    ax_k1.set_title('Frequenzbereich')
   
    #signal time domain
    d_t1 = dreieckTime(0,time_delta1[r1//2]*2) # triangle function: height=1, width=2s
    d_d1 = d_t1[np.arange(0,time_delta1[r1//2]*2,time_delta1[r1//2]//r1)] #diskrete signal
    
    zero_pad=N1-np.size(d_d1)
    
    if zero_pad < 0:
        s_d1 = d_d1[0:N1]
    else:
        #s_d1 = np.concatenate((np.zeros(zero_pad-zero_pad//2),d_d1,np.zeros(zero_pad//2)))
        s_d1 = np.concatenate((d_d1,np.zeros(zero_pad)))
        
    #plot/stem time domain
    rwth_plt.stem(ax_n1,np.arange(np.size(s_d1)),s_d1, label= '$S(n)$')
    ax_n1.plot(s_d1, label= '$s(t)$')
    
    #Design - Time domain: legend, axislabes and limits (garantees alignment of continous and diskrete)
    ax_t1.set_xlim([0,(np.size(s_d1)-1)/r1])
    ax_n1.set_xlim([0,np.size(s_d1)-1])    
    ax_n1.legend(loc='upper right')
    set_label_right(ax_n1,r'$\rightarrow n$')
    ax_n1.xaxis.set_label_coords(1.02, 0)
    set_label_right(ax_t1,r'$\rightarrow t$ in $[s]$')
    ax_t1.xaxis.set_label_coords(1.04, 1)
    
    #signal frequenz domain
    S_d1=np.fft.fft(s_d1)
    
    #plot/stem frequenz domain
    rwth_plt.stem(ax_k1,np.arange(np.size(S_d1)),1/r1*np.absolute(S_d1), label= '$S(k)$')
    ax_k1.plot(1/r1*np.absolute(S_d1), ls='--', label = '$S(f)$')
    
    #draw delta f 
    string1="$f_\Delta$ =\n" + str(np.round(r1/N1,2))
    ax_k1.annotate(string1,xy=(1.1, 0.02), xycoords='data',color = 'orange')#, xytext=(1.5,0.2), textcoords='data')
    
    con = ConnectionPatch(xyA=(1,0.0), xyB=(2,0.0), coordsA="data", coordsB="data",
                      axesA=ax_k1, axesB=ax_k1, color = 'orange')
    ax_k1.add_artist(con)
    
    #Design - Time domain: legend, axislabes and limits (garantees alignment of continous and diskrete)
    ax_f1.set_xlim([0,(np.size(S_d1)-1)*r1/N1])
    ax_k1.set_xlim([0,np.size(S_d1)-1])  
    set_label_right(ax_k1,r'$\rightarrow k$')
    ax_k1.xaxis.set_label_coords(1.02, 0)
    set_label_right(ax_f1,r'$\rightarrow f$ in $[Hz]$')
    ax_f1.xaxis.set_label_coords(1.04, 1)
    ax_k1.legend(loc='upper right')

    
    
#r1 = widgets.IntSlider(min = 2, max = 10, step = 2, value =4, description = '$r$:')
N1 = widgets.IntSlider(min = 8, max = 36 , step = 4, value = 16,  description = '$N$:')
    
widgets.interactive(update_plot, N1=N1) #r1=r1,

<div class="alert rwth-topic">

### Aufgabe 1.4: Diskrete Fourier-Transformation (DFT)

Ein Signal werde mit der Abtastfrequenz $r=8 \, \mathrm{kHz}$ abgetastet und mit der Diskreten Fouriert-Tansformation mit $N=8$ in den Frequenzbereich transformiert. Welchen Frequenzen entsprechen die einzelnen Fourier-Koeffizienten?

</div>

| k | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 |
| - | - | - | - | - | - | - | - | - |
| f | 




## Teil 2: Vorbereitung Python

In Ihrem bisherigen Studium haben Sie vor allem mit Matlab und C++ gearbeitet. In diesem Versuch wollen wir Ihnen Python (als Open-Source Alternative zu Matlab) näher bringen. Um die Umstellung von Matlab auf Python etwas leichter zu gestalten, haben wir ein interaktives Befehlsreferenzen Notebook erstellt, in dem Sie einige für den Versuch relevanten Befehle ausprobieren können. Bitte schauen Sie sich die Befehlsreferenzen gründlich an und bearbeiten Sie die nachfolgenden Aufgaben in der darunter liegenden Zelle. 

Alle notwendigen Bibliotheken sind in dem Notebook bereits eingebunden.

### Jupyter Quick Start

* Zum Ausführen aller Cells eines Notebooks: Im Menü: Cell <span class="fa-chevron-right fa"></span> Run All
* Zum Neustart (und erneutem Ausführen): <span class="fa-forward fa"></span>-Button
* Zum Ausführen einer einzelnen Cell: __Strg + Enter__ oder <span class="fa-step-forward fa"></span> Run-Button 

In [ ]:
from IPython.display import Video
Video("http://www.ient.rwth-aachen.de/cms/uploads/public/video/PTI-Videos/V_python.mp4", width=480, height=270 )

In [ ]:
import numpy as np 

<div class="alert rwth-topic">

### Aufgabe 2.1:

Erzeugen Sie ein Numpy-Array  __a__ mit den Werten: **[1,3,5,7,9]**. Wie lesen sie jetzt das erste Element im Array aus?
</div>

In [ ]:
#Lösen Sie Aufgabe 2.1 in diesem Feld

<div class="alert rwth-topic">

### Aufgabe 2.2 :

Erzeugen Sie nun eine Kopie von dem Numpy-Array __a__. Verbinden Sie nun **a** und die Kopie zu einem neuen Numpy-Array. 
Nutzen Sie dafür bitte die Funktion [np.hstack](https://docs.scipy.org/doc/numpy/reference/generated/numpy.hstack.html) oder [np.concatenate](https://docs.scipy.org/doc/numpy/reference/generated/numpy.concatenate.html).

</div>

In [ ]:
#Lösen Sie Aufgabe 2.2 in diesem Feld

<div class="alert rwth-topic">

### Aufgabe 2.3 :

Versuchen Sie mithilfe einer [For Schleife](https://wiki.python.org/moin/ForLoop) an das Array **a** 3 Mal das Numpy-Array __d = [4, 6, 8]__ anzuhängen, sodass Sie am Ende das Numpy-Array __[1, 3, 5, 7, 9, 4, 6, 8, 4, 6, 8, 4, 6, 8]__ herauskommt.


</div>

In [ ]:
#Lösen Sie Aufgabe 2.3 in diesem Feld

<div class="alert rwth-feedback">


    
# Feedback:

Liebe TeilnehmerInnen,

Wir würden uns freuen, wenn ihr am Ende kurz eure Meinung aufschreibt. Ihr könnt auf die dadrunter liegende Zelle zu greifen und eure Anmerkungen zu der Aufgabe (oder auch generelles) reinschreiben.


</div>

In [ ]:
feedback_name = 'Feeback V7 Vorbereitung'

questions = [
    {'id': 'likes', 'type': 'free-text-required', 'label': 'Das war gut:'}, 
    {'id': 'dislikes', 'type': 'free-text-required', 'label': 'Das könnte verbessert werden:'}, 
    {'id': 'misc', 'type': 'free-text', 'label': 'Was ich sonst noch sagen möchte:'}, 
    {'id': 'learning', 'type': 'scale', 'label' : 'Ich habe das Gefühl etwas gelernt zu haben.'},
    {'id': 'supervision', 'type': 'scale', 'label' : 'Die Betreuung des Versuchs war gut.'},
    {'id': 'script', 'type': 'scale', 'label' : 'Die Versuchsunterlagen sind verständlich.'},
]
rwth_feedback.RWTHFeedbackMail(feedback_name, questions, feedback_path='feedback.json', lang='de', mail_to="pti@ient.rwth-aachen.de");